In [ ]:
# !pip install line-bot-sdk flask
# !pip install gspread

In [ ]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("line_secret_key",'r',encoding="utf-8"))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
"""

準備好google 的credentials.json

並指定要操作的Spread Sheet

"""

import gspread
from google.oauth2.service_account import Credentials

scopes = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# 填寫金鑰檔案位置、名稱
credentials = Credentials.from_service_account_file('你的credential.json', 
                                                    scopes=scopes)

gc = gspread.authorize(credentials)

spread_sheet = gc.open_by_url('你的google sheet 網址')


In [ ]:
"""

用戶關注，收集用戶個資

"""

# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取得用戶個資
    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=user_profile.display_name + "您好！\n感謝您關注我們的機器人，我們將會收集您的個人資料。")
    )
    
    
    # 將用戶個人資料存入事先準備好的google sheet
    values_to_follow_wks = [event.source.user_id, 
                            event.timestamp,
                            user_profile.display_name,
                            user_profile.picture_url,
                            user_profile.status_message]

    follow_wks = spread_sheet.worksheet('FollowEventTable')
    follow_wks.append_row(values_to_follow_wks)

In [ ]:

'''

啟動Server

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')